# Homework II: Adversarial Search

In [1]:
from collections import namedtuple
import random

### TicTacToe Problem Class
Below is the problem and game state formulation we used for the TicTacToe problem.

In [2]:
GameState = namedtuple('GameState','to_move, utility, board, moves')

In [3]:
class TicTacToe:
    def __init__(self):
        moves = [(0,0),(0,1),(0,2),
                 (1,0),(1,1),(1,2),
                 (2,0),(2,1),(2,2)]
        self.initial = GameState(to_move = 'X', utility = 0, board = {}, moves = moves)
        
    def actions(self, state):
        # Returns legal moves (i.e. empty spaces)
        return state.moves
    
    def result(self, state, move):
        if move not in state.moves:
            return state # Illegal move
        
        # Board next turn
        board_next_turn = state.board.copy()
        board_next_turn[move] = state.to_move
        
        # Available moves next turn
        moves_next_turn = list(state.moves)
        moves_next_turn.remove(move)
        
        
        # Player next turn
        if state.to_move == 'X':
            player_next_turn = 'O'
        else:
            player_next_turn = 'X'
        
        # Utility next turn
        utility_next_turn = self.compute_utility(board_next_turn, move, state.to_move)
        
        return GameState(to_move = player_next_turn, utility = utility_next_turn, 
                         board = board_next_turn, moves = moves_next_turn)
                
    
    

    
    def compute_utility(self, board, move, player):
        if self.three_in_row(board, move, player):
            if player == 'X':
                return 1
            else:
                return -1
        else:
            return 0
  
    def terminal_test(self, state):
        return state.utility != 0 or len(state.moves) == 0

        
    def three_in_row(self, board, move, player):

        for (dx, dy) in ((0,1),(1,0),(1,1),(1,-1)):

            n = 0
            x, y = move
            while board.get((x,y)) == player:
                n = n + 1
                x = x + dx
                y = y + dy

            x, y = move
            while board.get((x,y)) == player:
                n = n + 1
                x = x - dx
                y = y - dy

            n = n - 1

            if n >= 3:
                return True

        return False


    
    def utility(self, state, player):
        if player == 'X':
            return state.utility
        else:
            return -state.utility
        
        
    def display(self, state):
        board = state.board
        for x in range(3):
            for y in range(3):
                print(board.get((x,y),'.'), end=' ')
            print()
            
    def play_game(self, player1, player2):
        state = self.initial
        players = [player1,player2]
        while True:
            for player in players:
                print('-----------------------')
                cur_player = state.to_move
                print(cur_player,'plays')
                move = player(self,state)
                state = self.result(state,move)
                self.display(state)
                if self.terminal_test(state):
                    print()
                    print('###########################')
                    if(self.utility(state,player) == 0):
                        print('Draw')
                    else:
                        print(cur_player,'wins!!!')
                    print('###########################')

                    return self.utility(state,self.initial.to_move)

### Player - Random Player and Minimax Player
Below are the functions that represents player agents that plays randomly and those that play with minimax algortihm. Examine the minimax carefully to understand how the algorithm works.

In [4]:
def random_player(game, state):
    if game.actions(state):
        return random.choice(game.actions(state))
    else:
        return None

In [5]:
def minmax_player(game, state):
    player = state.to_move
    
    def max_value(state):
        if game.terminal_test(state):
            return game.utility(state,player)
        v = -100
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a)))
        return v
            
    def min_value(state):
        if game.terminal_test(state):
            return game.utility(state,player)
        v = 100
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a)))
        return v
    
    best_score = -100
    best_action = None

    for a in game.actions(state):
        v = min_value(game.result(state,a))
        if v > best_score:
            best_score = v
            best_action = a
    return best_action
  

### A game
To play a game, we will create a TicTacToe object from the class we defined above. Below we create one minmax and one random player and we pass these players to `play_game(player1, player2)` method to make them play to each other.

In [6]:
ttt = TicTacToe()
p1 = minmax_player
p2 = random_player
ttt.play_game(p1,p2)

-----------------------
X plays
X . . 
. . . 
. . . 
-----------------------
O plays
X . . 
. O . 
. . . 
-----------------------
X plays
X X . 
. O . 
. . . 
-----------------------
O plays
X X . 
. O O 
. . . 
-----------------------
X plays
X X X 
. O O 
. . . 

###########################
X wins!!!
###########################


1

## Question 1 - Alpha Beta Pruning

Your first task is to implement alpha beta pruning algorithm to the function below. A part of the function has already been written for you. You need to implement within `max_value` and `min_value` functions that v is return only when it is greater than the beta or less than the alpha. 

A minmax player playing against a random player.

In [8]:
def alphabeta_player(game, state):
    player = state.to_move
    
    def max_value(state, alpha, beta):
        if game.terminal_test(state):
            return game.utility(state,player)
        v = -100
        for a in game.actions(state):                                                                      
            v = max(v, min_value(game.result(state, a), alpha, beta)) ###################
            if v >= beta:                                             ##implemented lines
                return v                                              ###################
            alpha = max(alpha, v)
        return v
            
    def min_value(state, alpha, beta):
        if game.terminal_test(state):
            return game.utility(state,player)
        v = 100
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a), alpha, beta)) ###################
            if v <= alpha:                                            ##implemented lines
                return v                                              ###################
            beta = min(beta, v)
        return v
    
    best_score = -100
    beta = 100
    best_action = None
    
    for a in game.actions(state):
        v = min_value(game.result(state,a), best_score, beta)
        if v > best_score:
            best_score = v
            best_action = a
    return best_action
  

After implementing alpha beta algorithm, you can make it play against a random player by running the code below.

In [9]:
ttt = TicTacToe()
p1 = alphabeta_player
p2 = random_player
ttt.play_game(p1,p2)

-----------------------
X plays
X . . 
. . . 
. . . 
-----------------------
O plays
X . . 
. . . 
. . O 
-----------------------
X plays
X . X 
. . . 
. . O 
-----------------------
O plays
X . X 
. . . 
. O O 
-----------------------
X plays
X X X 
. . . 
. O O 

###########################
X wins!!!
###########################


1

## Question 2 - Gomoku

Your second task is to formulate the problem for the game **Gomoku**. Below a Gomoku class has been defined but the code inside the class is exactly the same as tictactoe. You need to update the `moves`, `three_in_row` and `display` methods according to the available moves and winning conditions of the game respectively.

In [10]:
class Gomoku:
    def __init__(self):
                                     ###############
        moves=[]                     ##
        for i in range(15):          ## implemented
            for j in range(15):      ## lines
                moves.append((i, j)) ##
                                     ###############
    
        self.initial = GameState(to_move = 'X', utility = 0, board = {}, moves = moves)
        
    def actions(self, state):
        # Returns legal moves (i.e. empty spaces)
        return state.moves
    
    def result(self, state, move):
        if move not in state.moves:
            return state # Illegal move
        
        # Board next turn
        board_next_turn = state.board.copy()
        board_next_turn[move] = state.to_move
        
        # Available moves next turn
        moves_next_turn = list(state.moves)
        moves_next_turn.remove(move)
        
        
        # Player next turn
        if state.to_move == 'X':
            player_next_turn = 'O'
        else:
            player_next_turn = 'X'
        
        # Utility next turn
        utility_next_turn = self.compute_utility(board_next_turn, move, state.to_move)
        
        return GameState(to_move = player_next_turn, utility = utility_next_turn, 
                         board = board_next_turn, moves = moves_next_turn)
                
    
        
    def compute_utility(self, board, move, player):
        if self.five_in_row(board, move, player):
            if player == 'X':
                return 1
            else:
                return -1
        else:
            return 0
  
    def terminal_test(self, state):
        return state.utility != 0 or len(state.moves) == 0

        
    def five_in_row(self, board, move, player):
        
        for (dx, dy) in ((0,1), (1,0), (1,1), (1,-1)):
            n = 0
            x, y = move
            while board.get((x,y)) == player:
                n = n + 1
                x = x + dx
                y = y + dy

            x, y = move
            while board.get((x,y)) == player:
                n = n + 1
                x = x - dx
                y = y - dy

            n = n - 1
                                ######################################################################
            if n >= 5:          ## Evaluation criterion of n (and the name of the function) is changed
                return True     ## in order to check whether there are 5 X's or O's in a row or not.
                                ## This version of function clearly stops counting when there is 5 X's
                                ## or 5 O's, vertically, horizontally or diagonally. Then Gomoku()
                                ## returns utility value.
                                ######################################################################
        return False


    
    def utility(self, state, player):
        if player == 'X':
            return state.utility
        else:
            return -state.utility
        
        
    def display(self, state):
        board = state.board             ###############
        for x in range(15):             ## implemented
            for y in range(15):         ## lines
                                        ###############
                print(board.get((x,y),'.'), end=' ')
            print()
            
    def play_game(self, player1, player2):
        state = self.initial
        players = [player1,player2]
        while True:
            for player in players:
                print('-----------------------------')
                cur_player = state.to_move
                print(cur_player,'plays')
                move = player(self,state)
                state = self.result(state,move)
                self.display(state)
                if self.terminal_test(state):
                    print()
                    print('#############################')
                    if(self.utility(state,player) == 0):
                        print('Draw')
                    else:
                        print(cur_player,'wins!!!')
                    print('#############################')

                    return self.utility(state,self.initial.to_move)

After adapting the problem formulation for Gomoku, make two players play again each other. (If you could not implement alpha beta pruning use a minimax player instead of alpha beta player below.

In [ ]:
ggame = Gomoku()
p1 = alphabeta_player
p2 = random_player
ggame.play_game(p1,p2)

In [23]:
ggame = Gomoku()
p1 = random_player
p2 = random_player
ggame.play_game(p1,p2)

-----------------------------
X plays
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . . . X . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
-----------------------------
O plays
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . O . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . . . X . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . .

. . O . . . O . . . . . . . X 
X . . . . O X . . . . . . . . 
-----------------------------
O plays
. . . . . . . . . . . . . . . 
. . . O . . . . . . . . . . . 
. . . . . . . . . . . . . O . 
. . . . . . . . . . . . . . . 
X . . . . . . . . . . . . . . 
O . X . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . X . . . . . . . 
. O . . O . O . . . . . . . . 
. . . . . . . . . . . . . . . 
. X . . . . . . . X O . . . . 
. . . . . . . . . . . . . . . 
. . . . X . . . X . . . . . . 
. . O . . . O . . . . . . . X 
X . . . . O X . . . . . . . . 
-----------------------------
X plays
. . . . . . . . . . . . . . . 
. . . O . . . . . . . . . . . 
. . . . . . . . . . . . . O X 
. . . . . . . . . . . . . . . 
X . . . . . . . . . . . . . . 
O . X . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . X . . . . . . . 
. O . . O . O . . . . . . . . 
. . . . . . . . . . . . . . . 
. X . . . . . . . X O . . . . 
. . . . . . . . . . . . . . . 
. . . . X . . . X . . . .

. . . . X . . . X . . . . . . 
. . O . X . O . X . . O . . X 
X . . . . O X . . . . . . . . 
-----------------------------
O plays
O . . . . . . . . . . . . . O 
. . . O . . . . . . . X . . . 
. . . . . . . . . . . . . O X 
. . . . . . . . . . . . . . . 
X . . . . . . . . . . . . . . 
O . X . . . . . . X . X . . . 
. . . . . . . . . . . X . . . 
. O . . . . . X . . . . . . . 
. O . . O O O . . O . . . . . 
. . O . . . . O . . . O . . . 
. X . . . . . X . X O . X O X 
. . . . . . . . . . . . . . . 
. . . . X . . . X . . . . . . 
. . O . X . O . X . . O . . X 
X . . . . O X . . . . . . . . 
-----------------------------
X plays
O . . . . . . . . . . . . . O 
. . . O . . . . . . . X . . . 
. . . . . . . . . . . . . O X 
. . . . . . . . . . . . . . . 
X . . . . . . . . . . . . . . 
O . X . . . . . . X . X . . . 
. . . . . . . . . . . X . . . 
. O . . . . . X . . . . . . . 
. O . . O O O . . O . . . . . 
. . O . . . . O . . . O . . . 
. X . . . . . X . X O . X O X 
. . . . . . . . . . . . .

. . . O . . . . . . . . . O X 
. . . . O X . . . . . X . . . 
X X . . O . O . . . . . . . . 
O . X O . O . . . X . X X . . 
X . . . . . . . . . . X . X . 
. O . . . . . X . . . . . . . 
. O . . O O O . . O . . . . . 
. . O . . . . O . . . O . . . 
. X . . . . X X . X O O X O X 
. . . . . . . . X . . . . . . 
. . . X X . O . X . . . . X . 
. . O . X X O . X . . O O . X 
X . . . . O X . O . . . . . . 
-----------------------------
X plays
O . X . O . . . . . O . O . O 
. . . O X . . . . . . X . . . 
. . . O . . . . . . . . . O X 
. . . . O X . . . . . X . . . 
X X . . O . O . . . . . . . . 
O . X O . O . . . X . X X . . 
X . . . . X . . . . . X . X . 
. O . . . . . X . . . . . . . 
. O . . O O O . . O . . . . . 
. . O . . . . O . . . O . . . 
. X . . . . X X . X O O X O X 
. . . . . . . . X . . . . . . 
. . . X X . O . X . . . . X . 
. . O . X X O . X . . O O . X 
X . . . . O X . O . . . . . . 
-----------------------------
O plays
O . X . O . . . . . O . O . O 
. . . O X . . . . . . X .

. . O X X . O . X . . . O X . 
. O O . X X O . X X . O O . X 
X . . . . O X . O . . . . . X 
-----------------------------
O plays
O . X . O . . . . O O . O . O 
. . . O X O . . . . X X . . . 
O . . O . . . . . . . X . O X 
. . . . O X . . . . . X . . X 
X X O . O . O . X . . . X . . 
O . X O . O . . . X . X X X . 
X . . . . X . . . . O X X X X 
. O . . . . . X X . . O . . . 
. O . . O O O . . O . . . . . 
O . O . . . . O . . O O . . . 
. X . . . . X X . X O O X O X 
. . O . . . . . X . . . . . . 
. . O X X . O . X . . . O X . 
. O O . X X O . X X . O O . X 
X . . . . O X . O . . . . . X 
-----------------------------
X plays
O . X . O . . . . O O . O . O 
. . . O X O . . . . X X . . . 
O . . O . . . . . . . X . O X 
. . . . O X . . . . . X . . X 
X X O . O . O . X . . . X . . 
O . X O . O . . . X . X X X . 
X . . . . X . . . . O X X X X 
. O . . . . . X X . . O . . . 
. O . . O O O . . O . . . . . 
O . O . . . . O . . O O . . . 
. X . . . . X X . X O O X O X 
. . O . . . . . X . . . .

. . O X X . O X X . . . O X . 
. O O X X X O . X X . O O . X 
X . . . . O X . O . . . . . X 
-----------------------------
X plays
O . X . O . . . . O O . O . O 
. . . O X O . . . . X X O . X 
O . . O . . . . . . . X . O X 
. X . O O X . . . . O X . . X 
X X O . O . O . X X . . X . . 
O . X O X O O X . X . X X X . 
X . O . . X . O X . O X X X X 
. O . . . . . X X . X O . . . 
. O . . O O O . . O O . . . . 
O . O . . . . O . . O O . . . 
. X . . . . X X . X O O X O X 
X . O . O . . . X O . . . . . 
. . O X X . O X X . . . O X . 
. O O X X X O . X X . O O . X 
X . . . . O X . O . . . . . X 
-----------------------------
O plays
O . X . O . . . . O O . O . O 
. . . O X O . . . . X X O . X 
O . . O . . . . . . . X . O X 
. X . O O X . . . . O X . . X 
X X O . O . O . X X . . X . . 
O . X O X O O X . X . X X X . 
X . O . . X . O X . O X X X X 
. O . . . . . X X . X O . . . 
. O . . O O O . . O O . . . . 
O . O . . . . O . . O O . . . 
. X . . O . X X . X O O X O X 
X . O . O . . . X O . . .

1

## Question 3 - Connect 4

Your third task is to formulate the problem for the game **Connnect 4**. Below a Connect 4 class has been defined but the code inside the class is exactly the same as tictactoe. You need to update the `moves`, `three_in_row` and `display` methods according to the available moves and winning conditions of the game respectively. You must also  update `actions` function because in Connect 4, we can only play to the lowest empty row in each column.

In [67]:
class ConnectFour:
    def __init__(self):
                                      ################
        moves=[]                      ##
        for i in range(6):            ## implemented
            for j in range(7):        ## lines
                moves.append((i, j))  ##
                                      ################
        self.initial = GameState(to_move = 'X', utility = 0, board = {}, moves = moves)
        
    def actions(self, state):    ##########################################
        legal_actions = []       ## the rest of the function is implemented
        c0 = []                  ##########################################
        c1 = []
        c2 = []
        c3 = []          ####### Empty lists are created as representing columns.
        c4 = []
        c5 = []
        c6 = []
        for tuple in state.moves:    ### This for loop looks at all tuples in state.moves,
            if tuple[1] == 0:        ### then push the tuples, regarding to their second element(column),
                c0.append(tuple)     ### respectively into c0, c1, c2, c3, c4, c5, c6.
            elif tuple[1] == 1:      ### e.g. (5,0) should be pushed into c0, (5,6) should be
                c1.append(tuple)     ### pushed into c6 and so on...
            elif tuple[1] == 2:
                c2.append(tuple) 
            elif tuple[1] == 3: 
                c3.append(tuple) 
            elif tuple[1] == 4:
                c4.append(tuple)
            elif tuple[1] == 5:   
                c5.append(tuple)
            elif tuple[1] == 6:    
                c6.append(tuple)
        if len(c0) > 0:             
            legal_actions.append(max(c0, key=lambda x:x[0]))  ### Then this "if" section takes the maximum
        if len(c1) > 0:                                       ### value from the lists(c0, c1, c2, c3, c4, c5, c6)
            legal_actions.append(max(c1, key=lambda x:x[0]))  ### by looking the tuples first element(row),
        if len(c2) > 0:                                       ### and adds them into the legal_actions.
            legal_actions.append(max(c2, key=lambda x:x[0]))  ### e.g. (5,0) will be taken from c0 for the 
        if len(c3) > 0:                                       ### initial state, (5,1) from c1, (5,2) from c2,
            legal_actions.append(max(c3, key=lambda x:x[0]))  ### and so on... When (5,0) has "X" or "O" (4,0)
        if len(c4) > 0:                                       ### will be taken from c0.
            legal_actions.append(max(c4, key=lambda x:x[0]))
        if len(c5) > 0:
            legal_actions.append(max(c5, key=lambda x:x[0]))
        if len(c6) > 0:
            legal_actions.append(max(c6, key=lambda x:x[0]))
                   
        
        if state.to_move == 'X':                                #################################################
            print(f"Legal actions for p1 was: {legal_actions}") ## Added a print command to check the
        else:                                                   ## validity of my "actions" function with
            print(f"Legal actions for p2 was: {legal_actions}") ## my implementation of legal_actions.
        return legal_actions                                    ## It prints legal actions twice, because actions 
                                                                ## is called both by ConnectFour and random_player
                                                                ## function.
                                                                ##################################################
    
    def result(self, state, move):
        if move not in state.moves:
            return state # Illegal move
        
        # Board next turn
        board_next_turn = state.board.copy()
        board_next_turn[move] = state.to_move
        
        # Available moves next turn
        moves_next_turn = list(state.moves)
        moves_next_turn.remove(move)
        print(f"Move made this turn is: {move}")
        
        
        # Player next turn
        if state.to_move == 'X':
            player_next_turn = 'O'
        else:
            player_next_turn = 'X'
        
        # Utility next turn
        utility_next_turn = self.compute_utility(board_next_turn, move, state.to_move)
        
        return GameState(to_move = player_next_turn, utility = utility_next_turn, 
                         board = board_next_turn, moves = moves_next_turn)
                
    
        
    def compute_utility(self, board, move, player):
        if self.four_in_row(board, move, player):
            if player == 'X':
                return 1
            else:
                return -1
        else:
            return 0
  
    def terminal_test(self, state):
        return state.utility != 0 or len(state.moves) == 0

        
    def four_in_row(self, board, move, player):

        for (dx, dy) in ((0,1),(1,0),(1,1),(1,-1)):

            n = 0
            x, y = move
            while board.get((x,y)) == player:
                n = n + 1
                x = x + dx
                y = y + dy

            x, y = move
            while board.get((x,y)) == player:
                n = n + 1
                x = x - dx
                y = y - dy

            n = n - 1
                                         ####################################################
            if n >= 4:                   ## Evaluation criterion (n) is changed to 4 to check
                return True              ## whether X/O has four in a row or not. Stops 
                                         ## counting, when there are vertically, horizontally
                                         ## or diagonally 4 X's or 4 O's, and ConnectFour()
                                         ## returns utility value.
                                         ####################################################
        return False


    
    def utility(self, state, player):
        if player == 'X':
            return state.utility
        else:
            return -state.utility
        
        
    def display(self, state):
        board = state.board                           ##################
        for x in range(6):                     #########implemented line
            for y in range(7):                 #########implemented line
                print(board.get((x,y),'.'), end=' ')  ##################
            print()
            
    def play_game(self, player1, player2):
        state = self.initial
        players = [player1,player2]
        while True:
            for player in players:
                print('-----------------------')
                cur_player = state.to_move
                print(cur_player,'plays')
                move = player(self,state)
                state = self.result(state,move)
                self.display(state)
                if self.terminal_test(state):
                    print()
                    print('###########################')
                    if(self.utility(state,player) == 0):
                        print('Draw')
                    else:
                        print(cur_player,'wins!!!')
                    print('###########################')

                    return self.utility(state,player)

After implementing Connect 4, make two players play again each other. (If you could not implement alpha beta pruning use a minimax player instead of alpha beta player below.

In [ ]:
c4game = ConnectFour()
p1 = alphabeta_player
p2 = random_player
c4game.play_game(p1,p2)

In [68]:
c4game = ConnectFour()
p1 = random_player
p2 = random_player
c4game.play_game(p1,p2)

-----------------------
X plays
Legal actions for p1 was: [(5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6)]
Legal actions for p1 was: [(5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6)]
Move made this turn is: (5, 2)
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . X . . . . 
-----------------------
O plays
Legal actions for p2 was: [(5, 0), (5, 1), (4, 2), (5, 3), (5, 4), (5, 5), (5, 6)]
Legal actions for p2 was: [(5, 0), (5, 1), (4, 2), (5, 3), (5, 4), (5, 5), (5, 6)]
Move made this turn is: (5, 0)
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
O . X . . . . 
-----------------------
X plays
Legal actions for p1 was: [(4, 0), (5, 1), (4, 2), (5, 3), (5, 4), (5, 5), (5, 6)]
Legal actions for p1 was: [(4, 0), (5, 1), (4, 2), (5, 3), (5, 4), (5, 5), (5, 6)]
Move made this turn is: (5, 5)
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
O . X . . X . 
-----------------------
O plays
Legal actio

1